# CSP con Backtracking, Beam y Local Search
# Condiciones del problema:
### Se implementará 3 algoritmos diferentes de satisfacción de restricciones para resovler un problema de programación de exámenes para cuatro estudiantes que toman 7 exámenes diferentes. El problema implica calendarizar los exámenes para los estudiantes respetando diversas limitaciones y preferencias.
- Todos los exámenes deberán realizarse en días diferentes, concretamente, de lunes a miércoles.
- Ningún estudiante deberá tener más de un examen por día.
- Los estudiantes que toman el mismo curso no pueden tener exámenes programados para el mismo día

# Definición de variables y estructura del problema

In [19]:
import random
import time
from constraint import Problem

In [20]:
def setup_problem():
    exams = ['exam1', 'exam2', 'exam3', 'exam4', 'exam5', 'exam6', 'exam7']
    days = ['monday', 'tuesday', 'wednesday']
    student_exams = {
        'student1': ['exam1', 'exam2'],
        'student2': ['exam3', 'exam4'],
        'student3': ['exam5', 'exam6'],
        'student4': ['exam7']
    }

    problem = Problem()

    # Agregar las variables
    for exam in exams:
        problem.addVariable(exam, days)

    # Definir las restricciones
    for student, exams_list in student_exams.items():  # Cambio de nombre a exams_list
        for exam1 in exams_list:
            for exam2 in exams_list:
                if exam1 != exam2:
                    problem.addConstraint(lambda e1, e2: e1 != e2, (exam1, exam2))

    return exams, days, student_exams, problem


## Backtracking

In [21]:
def backtracking_search(exams, days, student_exams):
    problem = Problem()
    for exam in exams:
        problem.addVariable(exam, days)

    for student, student_exams in student_exams.items():
        for exam1 in student_exams:
            for exam2 in student_exams:
                if exam1 != exam2:
                    problem.addConstraint(lambda e1, e2: e1 != e2, (exam1, exam2))

    start_time = time.time()
    solutions = problem.getSolutions()
    end_time = time.time()
    return solutions, end_time - start_time


## Beam

In [22]:
def beam_search(student_exams, exams, days, beam_width):
    def is_valid_assignment(assignment, student_exams):
        for student, exams in student_exams.items():
            days = [assignment[exam] for exam in exams if exam in assignment]
            if len(days) != len(set(days)):  # Check for duplicate days
                return False
        return True

    def generate_successors(assignment, exams, days):
        successors = []
        day_usage = {day: sum(1 for d in assignment.values() if d == day) for day in days}

        for exam in exams:
            if exam not in assignment:
                sorted_days = sorted(days, key=lambda day: day_usage[day])
                for day in sorted_days:
                    new_assignment = assignment.copy()
                    new_assignment[exam] = day
                    successors.append(new_assignment)
                break
        return successors

    def heuristic(assignment, student_exams):
        return sum(1 for exam, day in assignment.items() if is_valid_assignment({exam: day}, student_exams))

    beam = [{}]
    solutions = []

    start_time = time.time()
    while beam:
        next_beam = []
        for assignment in beam:
            if len(assignment) == len(exams):
                if is_valid_assignment(assignment, student_exams):
                    solutions.append(assignment)
            else:
                successors = generate_successors(assignment, exams, days)
                next_beam.extend(successors)

        next_beam.sort(key=lambda a: heuristic(a, student_exams), reverse=True)
        beam = next_beam[:beam_width]

    end_time = time.time()
    return solutions, end_time - start_time

## Local Search

In [23]:
def local_search(exams, days, student_exams, max_iter):
    def generate_initial_solution():
        solution = {}
        for student, exams in student_exams.items():
            for exam in exams:
                while True:
                    day = random.choice(days)
                    if day not in solution.values():
                        solution[exam] = day
                        break
        return solution

    def generate_neighborhood(current_solution):
        neighbors = []
        for exam in exams:
            for day in days:
                if current_solution[exam] != day:
                    neighbor = current_solution.copy()
                    neighbor[exam] = day
                    neighbors.append(neighbor)
        return neighbors

    def heuristic(solution):
        conflicts = 0
        for student, exams in student_exams.items():
            days = [solution[exam] for exam in exams if exam in solution]
            if len(days) != len(set(days)):
                conflicts += 1
        return -conflicts  # Negative because we want to minimize conflicts

    def is_valid_neighbor(neighbor):
        for student, exams in student_exams.items():
            days = [neighbor[exam] for exam in exams if exam in neighbor]
            if len(days) != len(set(days)):
                return False
        return True

    current_solution = generate_initial_solution()
    current_cost = heuristic(current_solution)
    best_solution = current_solution.copy()
    best_cost = current_cost

    start_time = time.time()
    for _ in range(max_iter):
        neighbors = generate_neighborhood(current_solution)
        random.shuffle(neighbors)
        valid_neighbors = [neighbor for neighbor in neighbors if is_valid_neighbor(neighbor)]
        if not valid_neighbors:
            break
        best_neighbor = max(valid_neighbors, key=heuristic)
        best_neighbor_cost = heuristic(best_neighbor)
        if best_neighbor_cost <= best_cost:
            break
        else:
            current_solution = best_neighbor.copy()
            current_cost = best_neighbor_cost
            if current_cost > best_cost:
                best_solution = current_solution.copy()
                best_cost = current_cost

    end_time = time.time()
    return best_solution, end_time - start_time

# Main

In [24]:
exams, days, student_exams, problem = setup_problem()

## BACKTRACING SEARCH

In [25]:
backtracking_solutions, backtracking_time = backtracking_search(exams, days, student_exams)
print("Soluciones de Backtracking Search:")
for solution in backtracking_solutions:
    print("Alumno - Examen - Día:")
    for exam, day in solution.items():
        print(f"{[student for student, exams in student_exams.items() if exam in exams][0]} - {exam} - {day}")
    print()
print(f"Tiempo de ejecución de Backtracking Search: {backtracking_time} segundos\n")

AttributeError: 'list' object has no attribute 'items'

In [ ]:
backtracking_solutions, backtracking_time = backtracking_search(exams, days, student_exams)
print("Soluciones de Backtracking Search:")
for solution in backtracking_solutions:
    print("Alumno - Examen - Día:")
    for exam, day in solution.items():
        print(f"{[student for student, exams in student_exams.items() if exam in exams][0]} - {exam} - {day}")
    print()
print(f"Tiempo de ejecución de Backtracking Search: {backtracking_time} segundos\n")

AttributeError: 'list' object has no attribute 'items'

## BEAM

In [ ]:
beam_width = 3
beam_solutions, beam_time = beam_search(student_exams, exams, days, beam_width)
print("Soluciones de Beam Search:")
for solution in beam_solutions:
    print("Alumno - Examen - Día:")
    for exam, day in solution.items():
        print(f"{[student for student, exams in student_exams.items() if exam in exams][0]} - {exam} - {day}")
    print()
print(f"Tiempo de ejecución de Beam Search: {beam_time} segundos\n")

## LOCAL SEARCH

In [ ]:
max_iter = 1000
local_search_solution, local_search_time = local_search(exams, days, student_exams, max_iter)
print("Soluciones de Local Search:")
print("Alumno - Examen - Día:")
for exam, day in local_search_solution.items():
    print(f"{[student for student, exams in student_exams.items() if exam in exams][0]} - {exam} - {day}")
print(f"Tiempo de ejecución de Local Search: {local_search_time} segundos\n")